# **Import the necessary Packages**


In [2]:
pip install fredapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
#@title Packages Imported 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras as keras
from keras.layers import ELU, PReLU, LeakyReLU
from keras.utils.vis_utils import plot_model, model_to_dot

from google.colab import drive
from google.colab import files

from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, roc_curve, auc
from sklearn import metrics
import seaborn as sns
import datetime as dt
from fredapi import Fred
import os
import requests
from pprint import pprint


# **Step I: Load the data from pre-downloaded Excels**

In [5]:
#@title Data Obtain

drive.mount('/content/drive')
#path of data in google drive
pathbc = '/content/drive/MyDrive/Colab Notebooks/AI Data/BankruptedCompanies.csv'
pathhc1 = '/content/drive/MyDrive/Colab Notebooks/AI Data/HealthyCompanies1.csv'
pathhc2 = '/content/drive/MyDrive/Colab Notebooks/AI Data/HealthyCompanies2.csv'

#1. bankrupted companies data
bc_data = pd.read_csv(pathbc)
bc_data = bc_data.reset_index(drop=True)
bc_data.columns = ["Years/Attributions", "Year 5", "Year 4","Year 3","Year 2","Year 1"]
# change the antichronogical data into chronogical data by reversing the columns
bc_data = bc_data[bc_data.columns[::-1]]
bc_data.set_index("Years/Attributions", inplace = True)

#2. healthy companies data
#data1
hc_data1 = pd.read_csv(pathhc1)
hc_data1 = hc_data1.reset_index(drop=True)
hc_data1.columns = ["Years/Attributions", "Year 5", "Year 4","Year 3","Year 2","Year 1"]
#change the antichronogical data into chronogical data by reversing the columns
hc_data1 = hc_data1[hc_data1.columns[::-1]] 
#data2
hc_data2 = pd.read_csv(pathhc2)
hc_data2 = hc_data2.reset_index(drop=True)
hc_data2.columns = ["Years/Attributions", "Year 5", "Year 4","Year 3","Year 2","Year 1"]
# change the antichronogical data into chronogical data by reversing the columns
hc_data2 = hc_data2[hc_data2.columns[::-1]]
#concatenate
hc_data = pd.concat([hc_data1,hc_data2], ignore_index=True)
hc_data.set_index("Years/Attributions", inplace = True)

In [6]:
#@title Basic data conversion

#convert both data into numpy array
bc_data1 = bc_data.iloc[:,0:5].to_numpy()
hc_data3 = hc_data.iloc[:,0:5].to_numpy()
#split the data into blocks with each block possesses 28 attributes
bc_data2 = np.array(np.split(bc_data1.T,396,axis=1))
hc_data4 = np.array(np.split(hc_data3.T,39140,axis=1))

In [7]:
#@title Necessay Function Needed 

def confusionmatrixplot(confusionmatrix): #plot the confusion matrix given the array of data
  cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusionmatrix, display_labels = [False, True])
  cm_display.plot()
  plt.show()

def outlierremover(data,threshold): #remove the outliers with the specified threshold of our data
  mu = np.mean(data.T.reshape(data.shape[2],data.shape[0]*5),axis= 1)
  std = np.std(data.T.reshape(data.shape[2],data.shape[0]*5),axis= 1)
  z = (data - mu) / std
  mask = (z >= - threshold) & (z <= threshold)
  clean_data = data[np.all(mask, axis = (1,2))]
  return clean_data

# **Step II: Our way to extract those data through API key**

In [ ]:
#@title Extract US Market data 
#Get the US market data
# Set your FRED API key
api_key = 'a28dbdf390abae7716f8916162008096'

# Initialize the FRED API client
fred = Fred(api_key=api_key)

# Define the date range
end_date = dt.date(2022, 12, 31)
start_date = end_date - dt.timedelta(days=6*365)

# Download real GDP growth (annual, percent change) data
real_gdp_growth_data = fred.get_series('GDPC1', start_date, end_date)
rm = np.mean(np.split(np.array(real_gdp_growth_data),6),axis = 1)
real_gdp_growth_data = np.diff(rm) / rm[1:] * 100

# Download 10-year Treasury Bill rate data (risk-free rate proxy)
risk_free_rate_data = fred.get_series('GS10', start_date, end_date)
risk_free_rate_data = np.array(risk_free_rate_data.resample('Y').mean())[1:]

# Download unemployment rate (annual average) data
unemployment_rate_data = fred.get_series('UNRATE', start_date, end_date)
unemployment_rate_data = np.array(unemployment_rate_data.resample('Y').mean())[1:]

# Download inflation rate (annual, percent change) data using Consumer Price Index for All Urban Consumers (CPI-U)
inflation_data = fred.get_series('CPIAUCSL', start_date, end_date)
# inflation_data = inflation_data.pct_change().resample('Y').mean() *100
ri = np.mean(np.split(np.array(inflation_data),6),axis = 1)
inflation_data = np.diff(ri) / ri[1:] * 100

# Combine data into a single DataFrame
market_data = pd.DataFrame(columns = ['2018', '2019','2020','2021','2022'], index = ['Real GDP Growth','Inflation','Unemployment','10-y Risk Free'])
market_data.loc['Real GDP Growth'] = real_gdp_growth_data/100
market_data.loc['Inflation'] = inflation_data/100
market_data.loc['Unemployment'] = unemployment_rate_data/100
market_data.loc['10-y Risk Free'] = risk_free_rate_data/100
market_data

In [ ]:
#@title Extract fundamental data of corporations 

# Your Alpha Vantage API key
api_key ="VEUMZ77Y3NO13NKJ"

# companies' stock ticker
companies = ["IBM"] # change here

# API endpoints
base_url = "https://www.alphavantage.co/query?"
balance_sheet_url = f"{base_url}function=BALANCE_SHEET&apikey={api_key}"
income_statement_url = f"{base_url}function=INCOME_STATEMENT&apikey={api_key}"
stock_data_url = f"{base_url}function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&apikey={api_key}"

# Fetch and print the fundamental data for each company
try_data = pd.DataFrame()
for symbol in companies:
    print(f"Fetching data for {symbol}...")
    balance_sheet = requests.get(f"{balance_sheet_url}&symbol={symbol}").json()
    income_statement = requests.get(f"{income_statement_url}&symbol={symbol}").json()
    stock_data =  requests.get(f"{stock_data_url}&symbol={symbol}").json()

    data = balance_sheet['annualReports']
    bs = pd.DataFrame(data)
    bs.set_index('fiscalDateEnding', inplace=True)
    data1 = income_statement['annualReports']
    pl = pd.DataFrame(data1)
    pl.set_index('fiscalDateEnding', inplace=True)
    stock_data_df = pd.DataFrame(stock_data["Time Series (Daily)"]).T
    stock_data_df.index = pd.to_datetime(stock_data_df.index)
    stock_data_df["5. adjusted close"] = stock_data_df["5. adjusted close"].astype(float)
    stock_data_df["daily_returns"] = stock_data_df["5. adjusted close"].pct_change()

    # Filter stock data for the last 5 years
    five_years_ago = pd.Timestamp.now() - pd.DateOffset(years=6)
    filtered_stock_data_df = stock_data_df[stock_data_df.index >= five_years_ago]
    volatility = filtered_stock_data_df["daily_returns"].rolling(window=90).std() * np.sqrt(90)
    filtered_stock_data_df["90_day_volatility"] = volatility
    #Extract and compute all the 28 attributes from the financial statements
    yty_stock_price = filtered_stock_data_df['5. adjusted close'].resample('Y').last().astype('float')[:-1].pct_change()[1:].to_numpy()
    market_cap = bs['commonStockSharesOutstanding'].to_numpy().astype('float')*filtered_stock_data_df['5. adjusted close'].resample('Y').last().astype('float')[:-1][1:][::-1].to_numpy() / 1000000
    ninty_day_stock_std = filtered_stock_data_df['90_day_volatility'].resample('Y').last()[1:-1].astype('float')
    trading_volumn = filtered_stock_data_df['6. volume'].resample('Y').last().astype('float')[:-1][1:].to_numpy() / 1000
    total_assets = bs['totalAssets'].to_numpy().astype('float') / 1000000
    diluted_eps = pl['netIncome'].to_numpy().astype('float') / bs['totalShareholderEquity'].to_numpy().astype('float')
    liab_over_assets = bs['totalLiabilities'].to_numpy().astype('float') / bs['totalAssets'].to_numpy().astype('float')
    equity_over_assets = bs['totalShareholderEquity'].to_numpy().astype('float') / bs['totalAssets'].to_numpy().astype('float')
    cash_over_assets = bs['cashAndCashEquivalentsAtCarryingValue'].to_numpy().astype('float') / bs['totalAssets'].to_numpy().astype('float')
    current_ratio = bs['totalCurrentAssets'].to_numpy().astype('float') / bs['totalCurrentLiabilities'].to_numpy().astype('float')
    inventory_over_sales = bs['inventory'].to_numpy().astype('float') / pl['totalRevenue'].to_numpy().astype('float')
    netdebt_over_ebitda = (bs['longTermDebt'].to_numpy().astype('float') + bs['shortTermDebt'].to_numpy().astype('float') - bs['cashAndCashEquivalentsAtCarryingValue'].to_numpy().astype('float'))  / pl['ebitda'].to_numpy().astype('float')
    ap_over_sales = bs['currentAccountsPayable'].to_numpy().astype('float') / pl['totalRevenue'].to_numpy().astype('float')
    ar_over_sales = bs['currentNetReceivables'].to_numpy().astype('float') / pl['totalRevenue'].to_numpy().astype('float')
    roe = pl['netIncome'].to_numpy().astype('float') / bs['totalShareholderEquity'].to_numpy().astype('float')
    roa = pl['netIncome'].to_numpy().astype('float') / bs['totalAssets'].to_numpy().astype('float')
    sales_over_assets = pl['totalRevenue'].to_numpy().astype('float') / bs['totalAssets'].to_numpy().astype('float')
    net_income_margin = pl['netIncome'].to_numpy().astype('float') / pl['totalRevenue'].to_numpy().astype('float')
    ebit_over_sales = pl['ebit'].to_numpy().astype('float') / pl['totalRevenue'].to_numpy().astype('float')
    ebitda_over_sales = pl['ebitda'].to_numpy().astype('float') / pl['totalRevenue'].to_numpy().astype('float')
    ebitda_over_assets = pl['ebitda'].to_numpy().astype('float') / bs['totalCurrentAssets'].to_numpy().astype('float')
    debt_over_ebitda = (bs['longTermDebt'].to_numpy().astype('float') + bs['shortTermDebt'].to_numpy().astype('float')) / pl['ebitda'].to_numpy().astype('float')
    wc_over_assets = (bs['totalCurrentAssets'].to_numpy().astype('float') - bs['totalCurrentLiabilities'].to_numpy().astype('float')) / bs['totalAssets'].to_numpy().astype('float')
    price_to_book = filtered_stock_data_df['5. adjusted close'].resample('Y').last().astype('float')[:-1][1:].to_numpy()[::-1] / (bs['totalShareholderEquity'].to_numpy().astype('float') / bs['commonStockSharesOutstanding'].to_numpy().astype('float'))
    raw = np.concatenate((yty_stock_price,market_cap,ninty_day_stock_std,trading_volumn,total_assets,diluted_eps,liab_over_assets,equity_over_assets,cash_over_assets,current_ratio,inventory_over_sales,netdebt_over_ebitda,ap_over_sales,ar_over_sales,roe,roa,sales_over_assets,net_income_margin,ebit_over_sales,ebitda_over_sales,ebitda_over_assets,debt_over_ebitda,wc_over_assets,price_to_book))
    combined = np.split(raw,24)


In [ ]:
#@title Combine Data and Download as Excel

combined_data = pd.DataFrame(combined,columns = ['2022','2021','2020','2019','2018'])
combined_data = combined_data[combined_data.columns[::-1]]
whole_data = pd.concat([combined_data,market_data])
try_data = pd.concat([try_data,whole_data])
try_data.to_csv('outputUSCELLULAR.csv', encoding = 'utf-8-sig') 
files.download('outputUSCELLULAR.csv')

# **Step III: Conceptual Model and its performance**

In [ ]:
#@title Conceptual Model test
b = 500
accuracy = np.zeros((1,b))
f1 = np.zeros((1,b))
hcpower = np.zeros((1,b))
bcpower = np.zeros((1,b))
confusion = np.zeros((1,2,2))
for i in range(b):
    #randomly shuffle both datasets
    np.random.shuffle(bc_data2)
    np.random.shuffle(hc_data4)
    #Separating into training, validating and testing data sets:
    num_bc=396
    num_hc=39140
    #Define the train and test numbers of bc data
    n_train_bc = 220
    n_val_bc = 76
    n_test_bc = 100
    #Define the train and test numbers of hc data
    n_val_hc = int((n_val_bc*1))
    n_test_hc = int((n_test_bc*1))
    n_train_hc = num_bc - n_val_hc - n_test_hc
    #First select the test and validation dataset and keep it free from outlier remover(real-world senario)
    test_tuples = (list(zip(bc_data2[0:n_test_bc],np.ones(n_test_bc))) 
            + list(zip(hc_data4[0:n_test_hc],np.zeros(n_test_hc))))
    np.random.shuffle(test_tuples) #To mix the bankrupted and healty companies
    val_tuples = (list(zip(bc_data2[n_test_bc:n_test_bc+n_val_bc],np.ones(n_val_bc))) 
            + list(zip(hc_data4[n_test_hc:n_test_hc+n_val_hc],np.zeros(n_val_hc))))
    np.random.shuffle(val_tuples)
    #Then remove the outliers of the both bc and hc datasets and form the traning datasets
    bc_new = outlierremover(bc_data2[n_test_bc+n_val_bc:],threshold = 2000)
    hc_new = outlierremover(hc_data4[n_test_bc+n_val_bc:],threshold = 4.5)
    train_tuples = (list(zip(bc_new,np.ones(bc_new.shape[0]))) 
            + list(zip(hc_new,np.zeros(hc_new.shape[0]))))
    np.random.shuffle(train_tuples)
    #Extract the train,test,val datasets as np array from tuples
    (train_x, train_y) = [np.array(t).astype('float32') for t in zip(*train_tuples)]
    (val_x, val_y) = [np.array(t).astype('float32') for t in zip(*val_tuples)]
    (test_x, test_y) = [np.array(t).astype('float32') for t in zip(*test_tuples)]  
    mu_train = np.mean(train_x.T.reshape(train_x.shape[2],train_x.shape[0]*5),axis= 1)
    std_train = np.std(train_x.T.reshape(train_x.shape[2],train_x.shape[0]*5),axis= 1)
    train_x = (train_x - mu_train) / std_train
    test_x = (test_x - mu_train) / std_train
    val_x = (val_x - mu_train) / std_train 
    #Define the weight of loss function to focus more on the bankrupt companies(since bankrupt data is way less than the health data)
    class_weight_auto  = class_weight.compute_class_weight(
                                          class_weight = "balanced",
                                          classes = np.unique(train_y),
                                          y = train_y)
    class_weight_auto = {0:0.5, 1:120}
    #Define the callback to prevent overfitting by tracking the movement of val_loss
    es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    actual = test_y
    METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Recall(name='recall'),
    tf.keras.metrics.Precision(name='precision')
    ]
    model = keras.Sequential()
    model.add(keras.layers.LSTM(64, input_shape=(5,28), activation='tanh',dropout = 0.1))
    model.add(keras.layers.Dense(8, activation='PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = METRICS)
    model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 1)
    pre2 = np.rint(model.predict(test_x))
    f12 = f1_score(actual, pre2, average = 'weighted')
    confusion_matrix2 = metrics.confusion_matrix(actual, pre2)
    confusion[0]+=confusion_matrix2
    result1 = model.evaluate(test_x,test_y,batch_size=256,verbose = 1)
    bcpower[0][i] = result1[1]/100
    hcpower[0][i] = result1[3]/100
    accuracy[0][i] = result1[5]
    f1[0][i] = f12


In [ ]:
#@title Conceptual Model Performance Table and Confusion Matrix

accuracy_meanc = np.mean(accuracy,axis = 1)
max_accc = np.max(accuracy,axis = 1)
min_accc = np.min(accuracy,axis = 1)
acc_stdc = np.std(accuracy,axis = 1)
mean_bcpowerc = np.mean(bcpower,axis = 1)
mean_hcpowerc = np.mean(hcpower,axis = 1)

conceptual = pd.DataFrame(index = ['Conceptual LSTM'])
conceptual['Accuracy'] = np.round(accuracy_meanc,4)
conceptual['Recall BC'] = np.round(mean_bcpowerc,4)
conceptual['Recall HC'] = np.round(mean_hcpowerc,4)
conceptual['Max Accr'] = np.round(max_accc,4)
conceptual['Min Accr'] = np.round(min_accc,4)
conceptual['Accr Std'] = np.round(acc_stdc,4)

print(conceptual)
print(confusionmatrixplot(confusion[0]/b))

# **Step IV: Best Prediction model filter, selection and result presentation**

---



In [ ]:
#@title Prediction Model Filter 1 (10 models choose 3)

a = 1000
accuracy_store = np.zeros((10,a))
f1_store = np.zeros((10,a))
auc_store = np.zeros((10,a))
for i in range(a):
    np.random.shuffle(bc_data2)
    np.random.shuffle(hc_data4)
    #Separating into training, validating and testing data sets:
    proportion_train = 0.8
    proportion_val = 0.1
    num_bc=396
    num_hc=39140
    n_train_bc = 220
    n_val_bc = 76
    n_test_bc = 100
    n_val_hc = int((n_val_bc*1))
    n_test_hc = int((n_test_bc*1))
    n_train_hc = num_bc - n_val_hc - n_test_hc
    test_tuples = (list(zip(bc_data2[0:n_test_bc],np.ones(n_test_bc))) 
            + list(zip(hc_data4[0:n_test_hc],np.zeros(n_test_hc))))
    np.random.shuffle(test_tuples) #To mix the bankrupted and healty companies
    val_tuples = (list(zip(bc_data2[n_test_bc:n_test_bc+n_val_bc],np.ones(n_val_bc))) 
            + list(zip(hc_data4[n_test_hc:n_test_hc+n_val_hc],np.zeros(n_val_hc))))
    np.random.shuffle(val_tuples)

  #Then remove the outliers of the both bc and hc datasets
    def outlierremover(data,threshold):
      mu = np.mean(data.T.reshape(data.shape[2],data.shape[0]*5),axis= 1)
      std = np.std(data.T.reshape(data.shape[2],data.shape[0]*5),axis= 1)
      z = (data - mu) / std
      mask = (z >= - threshold) & (z <= threshold)
      clean_data = data[np.all(mask, axis = (1,2))]
      return clean_data
    bc_new = outlierremover(bc_data2[n_test_bc+n_val_bc:],threshold = 2000)
    hc_new = outlierremover(hc_data4[n_test_bc+n_val_bc:],threshold = 5)
    train_tuples = (list(zip(bc_new,np.ones(bc_new.shape[0]))) 
            + list(zip(hc_new,np.zeros(hc_new.shape[0]))))
    np.random.shuffle(train_tuples)

    (train_x, train_y) = [np.array(t).astype('float32') for t in zip(*train_tuples)]
    (val_x, val_y) = [np.array(t).astype('float32') for t in zip(*val_tuples)]
    (test_x, test_y) = [np.array(t).astype('float32') for t in zip(*test_tuples)]  
    mu_train = np.mean(train_x.T.reshape(train_x.shape[2],train_x.shape[0]*5),axis= 1)
    std_train = np.std(train_x.T.reshape(train_x.shape[2],train_x.shape[0]*5),axis= 1)
    train_x = (train_x - mu_train) / std_train
    test_x = (test_x - mu_train) / std_train
    val_x = (val_x - mu_train) / std_train 
    class_weight_auto  = class_weight.compute_class_weight(
                                          class_weight = "balanced",
                                          classes = np.unique(train_y),
                                          y = train_y)
    class_weight_auto = dict(enumerate(class_weight_auto))
    es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    #model1 GRU
    model = keras.Sequential()
    model.add(keras.layers.GRU(32,input_shape=(5,train_x.shape[2]),activation = 'tanh', dropout = 0.125,return_sequences=False,kernel_regularizer = keras.regularizers.L1(0.00007)))
    model.add(keras.layers.Dense(16, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(4, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid')) 
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x,test_y,batch_size=256)
    pre = model.predict(test_x)
    predicted = np.rint(model.predict(test_x))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[0][i] = test_acc
    f1_store[0][i] = f1
    auc_store[0][i] = roc_auc

    #model2 SimpleRNN
    model = keras.Sequential()
    model.add(keras.layers.SimpleRNN(128,input_shape=(5,train_x.shape[2]),activation = 'tanh', dropout = 0.125))
    model.add(keras.layers.Dense(32, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid')) 
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x,test_y,batch_size=256)
    pre = model.predict(test_x)
    predicted = np.rint(model.predict(test_x))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[1][i] = test_acc
    f1_store[1][i] = f1
    auc_store[1][i] = roc_auc

    #model3 LSTM
    model = keras.Sequential()
    model.add(keras.layers.LSTM(64,input_shape=(5,train_x.shape[2]),activation = 'tanh', dropout = 0.125))
    model.add(keras.layers.Dense(32, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid')) 
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x,test_y,batch_size=256)
    pre = model.predict(test_x)
    predicted = np.rint(model.predict(test_x))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[2][i] = test_acc
    f1_store[2][i] = f1
    auc_store[2][i] = roc_auc

    #model4 Dense
    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128,input_shape=(train_x.shape[2],),activation = 'tanh'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(64,activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(32, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid')) 
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x,test_y,batch_size=256)
    pre = model.predict(test_x)
    predicted = np.rint(model.predict(test_x))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[3][i] = test_acc
    f1_store[3][i] = f1
    auc_store[3][i] = roc_auc

    #model5 Conv-1D-v
    model = keras.Sequential()
    model.add(keras.layers.Conv1D(filters = 128,kernel_size = 5,input_shape=(28,5),activation = 'tanh',padding = 'same',kernel_regularizer = keras.regularizers.L1(0.00007)))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.MaxPooling1D(pool_size = 2))
    model.add(keras.layers.Conv1D(filters = 64,kernel_size = 4,activation = 'tanh',padding = 'same',kernel_regularizer = keras.regularizers.L2(0.00007)))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.MaxPooling1D(pool_size = 2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(32, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(4, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid'))
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x.transpose(0,2,1), train_y, validation_data=(val_x.transpose(0,2,1), val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x.transpose(0,2,1),test_y,batch_size=256)
    pre = model.predict(test_x.transpose(0,2,1))
    predicted = np.rint(model.predict(test_x.transpose(0,2,1)))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[4][i] = test_acc
    f1_store[4][i] = f1
    auc_store[4][i] = roc_auc

    #model6 Conv-1D-h
    model = keras.Sequential()
    model.add(keras.layers.Conv1D(filters = 256,kernel_size = 5,input_shape=(5,28),activation = 'tanh',padding = 'same'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.MaxPooling1D(pool_size = 2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid'))
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x,test_y,batch_size=256)
    pre = model.predict(test_x)
    predicted = np.rint(model.predict(test_x))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[5][i] = test_acc
    f1_store[5][i] = f1
    auc_store[5][i] = roc_auc

    #model7 BI-LSTM
    model = keras.Sequential()
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(50,input_shape=(5,train_x.shape[2]),activation = 'tanh', dropout = 0.1)))
    model.add(keras.layers.Dense(25, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(5, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(1,activation = 'sigmoid')) 
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x,test_y,batch_size=256)
    pre = model.predict(test_x)
    predicted = np.rint(model.predict(test_x))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[6][i] = test_acc
    f1_store[6][i] = f1
    auc_store[6][i] = roc_auc

    #model8 Bi-GRU
    model = keras.Sequential()
    model.add(keras.layers.Bidirectional(keras.layers.GRU(64,input_shape=(5,train_x.shape[2]),activation = 'tanh', dropout = 0.125)))
    model.add(keras.layers.Dense(32, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid')) 
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x,test_y,batch_size=256)
    pre = model.predict(test_x)
    predicted = np.rint(model.predict(test_x))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[7][i] = test_acc
    f1_store[7][i] = f1
    auc_store[7][i] = roc_auc

    #model 9 Conv lstm 1d
    model = keras.Sequential()
    model.add(keras.layers.Conv1D(filters = 64,kernel_size = 5,input_shape=(5,28),activation = 'tanh',padding = 'same'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.LSTM(128,activation = 'tanh', dropout = 0.25))
    model.add(keras.layers.Dense(32, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid'))
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x,test_y,batch_size=256)
    pre = model.predict(test_x)
    predicted = np.rint(model.predict(test_x))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[8][i] = test_acc
    f1_store[8][i] = f1
    auc_store[8][i] = roc_auc
  
    #model 10 Locally connected 1d
    model = keras.Sequential()
    model.add(keras.layers.LocallyConnected1D(filters = 256,kernel_size = 5,input_shape=(5,28),activation = 'tanh',padding = 'valid'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid'))
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])
    try1 = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    test_loss, test_acc = model.evaluate(test_x,test_y,batch_size=256)
    pre = model.predict(test_x)
    predicted = np.rint(model.predict(test_x))
    actual = test_y
    f1 = f1_score(actual, predicted, average = 'weighted')
    fpr,tpr,threshold = roc_curve(test_y, pre)
    roc_auc = auc(fpr,tpr)
    accuracy_store[9][i] = test_acc
    f1_store[9][i] = f1
    auc_store[9][i] = roc_auc

In [ ]:
#@title 10 Models Performance Comparison Table

# result performance presentation
auc_mean = np.mean(auc_store, axis = 1)
accuracy_mean = np.mean(accuracy_store,axis = 1)
f1_mean = np.mean(f1_store,axis = 1)
max_acc = np.max(accuracy_store,axis = 1)
min_acc = np.min(accuracy_store,axis = 1)
acc_std = np.std(accuracy_store,axis = 1)
max_f1 = np.max(f1_store,axis = 1)
min_f1 = np.min(f1_store,axis = 1)
max_auc = np.max(auc_store,axis = 1)
min_auc = np.min(auc_store,axis = 1)

compare = pd.DataFrame(index = ['GRU','SimpleRNN','LSTM','Dense','Conv 1D Vert','Conv 1D Hori','Bi-LSTM','Bi-GRU','Conv LSTM 1D','LocallyConv1D'])
compare['Accuracy'] = np.round(accuracy_mean,4)
compare['F1 Score'] = np.round(f1_mean,4)
compare['AUC Value'] = np.round(auc_mean,4)
compare['Max Accr'] = np.round(max_acc,4)
compare['Min Accr'] = np.round(min_acc,4)
compare['Accr Std'] = np.round(acc_std,4)
compare

In [ ]:
#@title Best Prediction Models selection and Performance presentations 

b = 1000
accuracy = np.zeros((3,b))
f1 = np.zeros((3,b))
hcpower = np.zeros((3,b))
bcpower = np.zeros((3,b))
confusion = np.zeros((3,2,2))
for i in range(b):
    np.random.shuffle(bc_data2)
    np.random.shuffle(hc_data4)
    num_bc=396
    num_hc=39140
    n_train_bc = 220
    n_val_bc = 76
    n_test_bc = 100
    n_val_hc = int((n_val_bc*1))
    n_test_hc = int((n_test_bc*1))
    n_train_hc = num_bc - n_val_hc - n_test_hc
    test_tuples = (list(zip(bc_data2[0:n_test_bc],np.ones(n_test_bc))) 
            + list(zip(hc_data4[0:n_test_hc],np.zeros(n_test_hc))))
    np.random.shuffle(test_tuples) 
    val_tuples = (list(zip(bc_data2[n_test_bc:n_test_bc+n_val_bc],np.ones(n_val_bc))) 
            + list(zip(hc_data4[n_test_hc:n_test_hc+n_val_hc],np.zeros(n_val_hc))))
    np.random.shuffle(val_tuples)
    bc_new = outlierremover(bc_data2[n_test_bc+n_val_bc:],threshold = 2000)
    hc_new = outlierremover(hc_data4[n_test_bc+n_val_bc:],threshold = 4.5)
    train_tuples = (list(zip(bc_new,np.ones(bc_new.shape[0]))) 
            + list(zip(hc_new,np.zeros(hc_new.shape[0]))))
    np.random.shuffle(train_tuples)
    (train_x, train_y) = [np.array(t).astype('float32') for t in zip(*train_tuples)]
    (val_x, val_y) = [np.array(t).astype('float32') for t in zip(*val_tuples)]
    (test_x, test_y) = [np.array(t).astype('float32') for t in zip(*test_tuples)]  
    mu_train = np.mean(train_x.T.reshape(train_x.shape[2],train_x.shape[0]*5),axis= 1)
    std_train = np.std(train_x.T.reshape(train_x.shape[2],train_x.shape[0]*5),axis= 1)
    train_x = (train_x - mu_train) / std_train
    test_x = (test_x - mu_train) / std_train
    val_x = (val_x - mu_train) / std_train 
    # class_weight_auto  = class_weight.compute_class_weight(
    #                                       class_weight = "balanced",
    #                                       classes = np.unique(train_y),
    #                                       y = train_y)
    class_weight_auto = {0:0.5, 1:120}
    es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    actual = test_y
    METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Recall(name='recall'),
    tf.keras.metrics.Precision(name='precision')
    ]

    # #Final Model1: Conv 1D verticle Model
    model = keras.Sequential()
    model.add(keras.layers.Conv1D(filters = 128,kernel_size = 5,input_shape=(28,5),activation = 'tanh',padding = 'same',kernel_regularizer = keras.regularizers.L1(0.00007)))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.MaxPooling1D(pool_size = 2))
    model.add(keras.layers.Conv1D(filters = 64,kernel_size = 4,activation = 'tanh',padding = 'same',kernel_regularizer = keras.regularizers.L2(0.00007)))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.MaxPooling1D(pool_size = 2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(32, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(4, activation = 'PReLU'))
    model.add(keras.layers.Dropout(0.125))
    model.add(keras.layers.Dense(1,activation = 'sigmoid'))
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = METRICS)
    try1 = model.fit(train_x.transpose(0,2,1), train_y, validation_data=(val_x.transpose(0,2,1), val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    pre1 = np.rint(model.predict(test_x.transpose(0,2,1)))
    f11 = f1_score(actual, pre1, average = 'weighted')
    confusion_matrix = metrics.confusion_matrix(actual, pre1)
    confusion[0]+=confusion_matrix
    result = model.evaluate(test_x.transpose(0,2,1),test_y,batch_size=256,verbose = 0)
    bcpower[0][i] = result[1]/100
    hcpower[0][i] = result[3]/100
    accuracy[0][i] = result[5]
    

    #Final Model2: BI-LSTM
    model1 = keras.Sequential()
    model1.add(keras.layers.Bidirectional(keras.layers.LSTM(50,input_shape=(5,train_x.shape[2]),activation = 'tanh', dropout = 0.1,kernel_regularizer = keras.regularizers.L1(0.00007))))
    model1.add(keras.layers.Dense(25, activation = 'PReLU'))
    model1.add(keras.layers.Dropout(0.2))
    model1.add(keras.layers.Dense(5, activation = 'PReLU'))
    model1.add(keras.layers.Dropout(0.2))
    model1.add(keras.layers.Dense(1,activation = 'sigmoid')) 
    model1.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = METRICS)
    try2 = model1.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    pre2 = np.rint(model1.predict(test_x))
    f12 = f1_score(actual, pre2, average = 'weighted')
    confusion_matrix2 = metrics.confusion_matrix(actual, pre2)
    confusion[1]+=confusion_matrix2
    result1 = model1.evaluate(test_x,test_y,batch_size=256,verbose = 0)
    bcpower[1][i] = result1[1]/100
    hcpower[1][i] = result1[3]/100
    accuracy[1][i] = result1[5]

    #Final Model3: GRU
    model2 = keras.Sequential()
    model2.add(keras.layers.GRU(32,input_shape=(5,train_x.shape[2]),activation = 'tanh', dropout = 0.125,return_sequences=False,kernel_regularizer = keras.regularizers.L1(0.00007)))
    model2.add(keras.layers.Dense(16, activation = 'PReLU'))
    model2.add(keras.layers.Dropout(0.125))
    model2.add(keras.layers.Dense(4, activation = 'PReLU'))
    model2.add(keras.layers.Dropout(0.125))
    model2.add(keras.layers.Dense(1,activation = 'sigmoid')) 
    model2.compile(loss= tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(), metrics = METRICS)
    try3 = model2.fit(train_x, train_y, validation_data=(val_x, val_y),epochs = 40, batch_size = 1024, callbacks = [es_callback],class_weight=class_weight_auto,verbose = 0)
    pre3 = np.rint(model2.predict(test_x))
    f13 = f1_score(actual, pre3, average = 'weighted')
    confusion_matrix3 = metrics.confusion_matrix(actual, pre3)
    confusion[2]+=confusion_matrix3
    result2 = model2.evaluate(test_x,test_y,batch_size=256,verbose = 0)
    bcpower[2][i] = result2[1]/100
    hcpower[2][i] = result2[3]/100
    accuracy[2][i] = result2[5]

In [ ]:
#@title 3 Candidate Models Performance Comparison Table

accuracy_mean1 = np.mean(accuracy,axis = 1)
max_acc1 = np.max(accuracy,axis = 1)
min_acc1 = np.min(accuracy,axis = 1)
acc_std1 = np.std(accuracy,axis = 1)
mean_bcpower = np.mean(bcpower,axis = 1)
mean_hcpower = np.mean(hcpower,axis = 1)

compare1 = pd.DataFrame(index = ['Conv 1D Vert','Bi-LSTM','GRU'])
compare1['Accuracy'] = np.round(accuracy_mean1,4)
compare1['Recall BC'] = np.round(mean_bcpower,4)
compare1['Recall HC'] = np.round(mean_hcpower,4)
compare1['Max Accr'] = np.round(max_acc1,4)
compare1['Min Accr'] = np.round(min_acc1,4)
compare1['Accr Std'] = np.round(acc_std1,4)
compare1

# **Reference**
[1] Juan Pablo Bonfrisco Ayala, Javier de Vicente Moreno. (2019). *APPLYING DEEP LEARNING TECHNIQUES TO PREDICT THE BANKRUPTCY OF AN ENTERPRISE*